In [46]:
import numpy as np
well_las_attr_dict=np.load('well_las_attr_dict.npy')[0]
# well_las_dict=np.load('well_las_dict.npy')[0]
# well_las_attr_dict

In [31]:
# awell=well_las_attr_dict['BRHL-56']
# lwithgr={}
# lwithoutgr={}

# for l in awell:
# #     print(awell[l]['depthrange'],', '.join(awell[l]['categories']))
#     if 'GR' in awell[l]['categories']:
#         lwithgr[l]=awell[l]
#     else:
#         lwithoutgr[l]=awell[l]
# print_logs(lwithgr)  
# print('*************************************')
# print_logs(lwithoutgr)  


# sorted_well,minmax,darray=sort_ranges(awell)
# print_logs(sorted_well)
# print(logs)
# sort_indx
# print(logs[sort_indx])
def print_logs(awell):
    for l in awell:
        print(awell[l]['depthrange'],', '.join(awell[l]['categories']))
def printSuits(suits):
    for i in suits:
        print('   ',i)
        for l in suits[i]:
            print('     ',l,suits[i][l]['depthrange'],', '.join(suits[i][l]['categories']))
def sort_ranges(awell):
#     minranges=np.array([min(awell[l]['depthrange']) for l in awell])
#     maxranges=np.array([max(awell[l]['depthrange']) for l in awell])
    
    darray=np.array([(min(awell[l]['depthrange']),max(awell[l]['depthrange'])) for l in awell])
    minmaxs=min(darray[:,0]),max(darray[:,1])
    logs=np.array(list(awell.keys()))
    sort_indx=np.argsort(darray[:,0])
    # sort_indx=sort_indx[::-1]
    sorted_well={}
    for l in logs[sort_indx]:
        sorted_well[l]=awell[l]
    return sorted_well,minmaxs,darray[sort_indx]
# def removeRepetedRuns(awell):
import matplotlib.pyplot as plt
def get_rangearray(awell):
    return np.array([(min(awell[l]['depthrange']),max(awell[l]['depthrange'])) for l in awell])

# plt.hist(minranges,20)
# maxranges-minranges,minmax
rangelength=100
def get_closeValuesIndx(value,val_array,rangelength):
    indexes=[]
    for i,v in enumerate(val_array):
        if (v<=value+rangelength)&(v>=value-rangelength):
            indexes.append(i)
    return indexes
def getLfileWIndex(awell,indexes):
    resultwell={}
    for fkey in np.array(list(awell.keys()))[indexes]:
        resultwell[fkey]=awell[fkey]
    return resultwell
def getLoglist(awell):
    logs=[]
    for l in awell:
        logs.extend(awell[l]['categories'])
    return np.unique(logs)
def getIndxGroups(valuearray,rangelength=200):
    temparray=valuearray
    indxarray=np.arange(0,len(valuearray))
#     print(indxarray)
    indxgroups=[]
    v=temparray[0]+0.00001
#     print(v,temparray,rangelength)
    while v:
        vindxes=get_closeValuesIndx(v,temparray,rangelength)
#         print('********************** \n',vindxes)
        v= np.mean(temparray[vindxes])
        vindxes=get_closeValuesIndx(v,temparray,rangelength)
#         print(vindxes)
        indxgroups.append(indxarray[vindxes])
#         print(temparray[vindxes])
    #     del temparray[vindxes]
        temparray=np.delete(temparray, vindxes)
        indxarray=np.delete(indxarray, vindxes)
    #     print(temparray)
        if len(temparray)>1:
            v=temparray[0]
        else:
            break
    return indxgroups
# getIndxGroups(dranges[:,0]) 

In [32]:
# j=[1,2,34,4]
# j.pop(2)
# j
# # bwell=awell.copy()
# # del bwell['jagun-1_wire.las']
class Logset():
    def __init__(self,singlewell):
        self.well=singlewell
        self.key=list(singlewell.keys())[0]

        self.logs=self.well[self.key]['categories']
        self.mindepth=min(self.well[self.key]['depthrange'])
        self.maxdepth=max(self.well[self.key]['depthrange'])
    def isRangesubset(self,minmaxes):
        if (minmaxes[0]>self.mindepth )& (minmaxes[1]<self.maxdepth):
            return True
        else:
            return False
    def isSubset(self,otherwell):
        owell=Logset(otherwell)
        if self.isRangesubset([owell.mindepth,owell.maxdepth]):
            for ol in owell.logs:
                if ol not in self.logs:
                    return False
        else:
            return False
        return True
# def removeSubsets(awell):       
# #     sorted_well,minmaxs=sort_ranges(awell)
#     reswell=awell.copy()
#     keys=list(awell.keys())
#     varkeys=keys.copy()
    
#     for i in range(len(keys)):
#         print(keys[i])
#         if keys[i] in awell.keys():
#             alset=Logset({keys[i]:awell[keys[i]]})
#     #         varkeys.pop(i)
#             for vk in varkeys:            
#                 if alset.isSubset({vk:awell[vk]}):
#                     print('      ',vk,end=', ')
#                     del reswell[vk] 
#                     print(len(reswell.keys()))
#             varkeys=list(reswell.keys())
#             awell=reswell.copy()  
#     return reswell



In [47]:
def getExclusivekeys(key,varkeys,awell):
    if key in awell.keys():
        alset=Logset({key:awell[key]})
        for vk in varkeys:            
            if alset.isSubset({vk:awell[vk]}):
                del awell[vk] 
        varkeys=list(awell.keys())
    return varkeys,awell
def removeSubsets(awell):       
    keys=list(awell.keys())
    varkeys=keys.copy()    
    for key in keys:
        varkeys,awell=getExclusivekeys(key,varkeys,awell)
#     print(varkeys)
    finalkeys=varkeys.copy()
#     print('final keys: ',len(finalkeys))
    for key in varkeys[::-1]:
        finalkeys,awell=getExclusivekeys(key,finalkeys,awell)
#     print('final keys: ',len(finalkeys))
    return awell

def global_logavailability(global_loglist,alas):
    if len(global_loglist)==0:
        return 0,[]
    lcount=0
    not_inGlobal=[]
    for log in global_loglist:
        if log in alas['categories']:
            lcount +=1
        else:
            not_inGlobal.append(log)
    return lcount/len(global_loglist),not_inGlobal

def similarityIndex(alas,blas):
    lcount=0
    for log in alas['categories']:
        if log in blas['categories']:
            lcount +=1
    return lcount/len(alas['categories'])
def getValidloginRun(partwell,list_availability,list_missed_logs):
    local_indx=np.argsort(np.array(list_availability))
    local_indx=local_indx[::-1]
    sorted_la=list_availability[local_indx]
    usorted_la=np.unique(sorted_la)
#     print('usorted la: ',usorted_la)
    lookindex=[]
    for i in range(0,len(sorted_la)):
        if sorted_la[i]==usorted_la[-1]:
            lookindex.append(i)
#     print('indexes found: ',lookindex)
    lookwell=getLfileWIndex(partwell,local_indx[lookindex])
    look_missed_logs=list_missed_logs[local_indx[lookindex]]
    validLFindx=np.argmax(np.diff(get_rangearray(lookwell)))
    return getLfileWIndex(lookwell,[validLFindx]),look_missed_logs[validLFindx]


def suitify(awell):
    global_loglist=getLoglist(awell)
    sorted_well,minmax,dranges=sort_ranges(awell)
    pseudo_suitindexes=getIndxGroups(dranges[:,0])

#     for indgroup in pseudo_suitindexes:
#         print_logs(getLfileWIndex(sorted_well,indgroup))
#         print('********************************************************')
#     # for l in awell:
    suits={}
    for i,indgroup in enumerate(pseudo_suitindexes):
        partwell=getLfileWIndex(sorted_well,indgroup)
        list_availability=[]
        list_missed_logs=[]
        suits[i]={}
        for l in partwell:
            availability,missed_logs=global_logavailability(global_loglist,partwell[l])
            list_availability.append(availability)
            list_missed_logs.append(missed_logs)
        avalawell,miss_log_list=getValidloginRun(partwell,np.array(list_availability),np.array(list_missed_logs))
        miss_log_avail=[]
        if len(miss_log_list)>0:
            for l in partwell:        
                availability,missed_logs=global_logavailability(miss_log_list,partwell[l])        
                miss_log_avail.append(availability)
        miss_log_avail=np.array(miss_log_avail)
        miss_indx=np.argsort(miss_log_avail)
        misavalawell=[]
    #     print(list(avalawell.keys())[0])
        key=list(avalawell.keys())[0]
        suits[i][key]=avalawell[key]
        if miss_log_avail[miss_indx[-1:]]>0:
            misavalawell=getLfileWIndex(partwell,miss_indx[-1:])
            key=list(misavalawell.keys())[0]
            suits[i][key]=misavalawell[key]
    #     print(avalawell)      
    #     print(np.array(miss_log_avail)[miss_indx])
    #     print(misavalawell)
    #     print('****************************************************************')
    return suits    

# for indgroup in pseudo_suitindexes:
#     print_logs(getLfileWIndex(sorted_well,indgroup))
wellsuits={}
for i,well in enumerate(well_las_attr_dict):
#     if i==9:
#         print(i,well)
#         for lf in well_las_attr_dict[well]:
#             # print(well_las_attr_dict[well][lf]['depthrange'])
#                 # well_las_attr_dict[well][lf]={}
#             print('      {:>25} : {:4.1f}-{:4.1f} , {:>15}'.format(lf,min(well_las_attr_dict[well][lf]['depthrange']),max(well_las_attr_dict[well][lf]['depthrange']),
#             ', '.join(well_las_attr_dict[well][lf]['categories'])))
        print(i,well)
        awell=well_las_attr_dict[well]

        awell=removeSubsets(awell)
#         print_logs(awell)
        suits=suitify(awell)
        wellsuits[well]=suits
        printSuits(suits)
np.save('wellsuits.npy',wellsuits)

0 AMGR-8A
    0
      10034448_E-AMGR-8A_T_150912_0.las (800.1, 2230.5264) CAL, DTC, GR, RHOB, NPHI, LLD, LLS, SP
1 AMGURI-1
    0
      Amguri1_SDL_DSN_CSNG_3798-4075.las (1881.2, 2960.4) CAL, GR, SGR, RHOB, NPHI, PE
    1
      DLL_MAIN.las (2913.6, 3908.5) CAL, GR, LLD, LLS, MSFL, SP
      SDL_MAIN.las (2921.5, 3909.0) CAL, GR, RHOB, NPHI, PE
2 BARBAM-1
    0
      Barbam-1_CDL1.las (1509.979, 235.001) CAL, GR, RHOB, NPHI
      Barbam-1_DIL1.las (1506.017, 242.164) GR, LLD, LLS, SP
    1
      Barbam-1_DLL2.las (3490.7, 1422.9) CAL, GR, LLD, LLS, MSFL, SP
      Barbam-1_SDL2.las (3491.5, 1417.8) CAL, GR, RHOB, NPHI, PE
    2
      Barbam-1_DLL3.las (3435.8, 4034.5) CAL, GR, LLD, LLS, MSFL, SP
      Barbam-1_SDL3.las (3440.5, 4116.7) CAL, GR, RHOB, NPHI, PE
3 BBJA-1
    0
      5148922_E-BBJA-1_T_140206_0.las (23.33, 2538.3872) CAL, DTC, GR, RHOB, NPHI, PE, LLD, LLS, MSFL
4 BOGAPANI-1
    0
      bogapani1_sonic1.las (250.0, 4810.0) CAL, SP
    1
      bogapani1_sonic2.las (4800.0, 1

C:\Users\70000612\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(1675.0, 3624.958) GR, LLD, LLS, MSFL, SP
52 MORAN-116
    0
      Moran116_DLL_LDL_INCL1.las (1615.44, 139.9032) CAL, RHOB, NPHI, PE, LLD, LLS, MSFL, SP
      Moran116_WSTT1.las (141.42, 1109.42001) CAL, DTC, GR
    1
      Moran116_DLL2.las (3531.1001, 1514.70007) CAL, GR, LLD, LLS, MSFL, SP
      Moran116_SDL2.las (3529.6001, 1568.80007) CAL, GR, RHOB, NPHI, PE
    2
      Moran116_DLL3.las (4284.2002, 3385.50018) CAL, GR, LLD, LLS, MSFL, SP
      Moran116_SDL3.las (3840.80005, 3492.20004) CAL, GR, RHOB, NPHI, PE
53 MRNI-2
    0
      2208490_E-MRNI-2_T_145419_0.las (50.1396, 619.9632) SP
54 NAGN-1
    0
      19642845_E_NAGN_1.las (9.134, 2483.348) CAL, DTC, GR, RHOB, NPHI, LLD, LLS, MSFL, SP
55 NAMDANG-1
    0
      NAMDANG1_17.5IN_COMPOSITE_FINAL_MAIN2.LAS (155.4816, 743.9816) CAL, DTC, GR, RHOB, LLS, SP
    1
      NAMDANG1_12.25IN_COMPOSITE_FINAL_MAIN2.LAS (735.9396, 1637.4396) CAL, DTC, GR, RHOB, LLS
    2
      NAMDANG1_85IN_SECT_COMPOSITE_MAIN_FINAL2.las (1560.4561, 2391.956

In [58]:
# wellsuits[list(wellsuits.keys())[2]]
import os
os.path.isfile(wellsuits['BARBAM-1'][0]['Barbam-1_CDL1.las']['path']) 
# for i,well in enumerate(well_las_attr_dict):
#     if i==3:
#         print(i,well)
#         for lf in well_las_attr_dict[well]:
#             # print(well_las_attr_dict[well][lf]['depthrange'])
#                 # well_las_attr_dict[well][lf]={}
#             print('      {:>25} : {:4.1f}-{:4.1f} , {:>15}'.format(lf,min(well_las_attr_dict[well][lf]['depthrange']),max(well_las_attr_dict[well][lf]['depthrange']),
#             ', '.join(well_las_attr_dict[well][lf]['categories'])))

True

In [93]:
# def dist(a,b,ax=0):
#     return np.linalg.norm(a-b,axis=ax)
# ngroups=4
# # v=0
# # get_closeValuesIndx(v,dranges[:,0],100)
# # from copy import deepcopy
# # Loop will run till the error becomes zero
# X=np.array([dranges[:,0]]*ngroups).T
# C=np.random.choice(dranges[:,0], ngroups)
# C_old=np.zeros(ngroups)
# Cs=C.copy()
# clusters=np.zeros(len(X))
# ithrun=0
# error=dist(C,C_old,None)
# # for j in range(0,1):
# while error > 0.1:
#     ithrun +=1
#     # Assigning each value to its closest cluster
#     print('{}th run'.format(ithrun))
#     for i in range(len(X)):
#         distances = dist(X[i], C)
#         print(distances)
#         print('distances')
#         cluster = np.argmin(distances)
#         clusters[i] = cluster
#     # Storing the old centroid values
#     C_old = C.copy()
#     print(clusters)
#     # Finding the new centroids by taking the average value
#     for i in range(ngroups):
#         points = [X[j] for j in range(len(X)) if clusters[j] == i]
#         C[i] = np.mean(points, axis=0)
#     error = dist(C, C_old, None)
# #     Cs=np.append(Cs,C, axis=0)
#     Cs=np.concatenate((Cs,C), axis=0)
